In [6]:
from n_grams import *
import pandas as pd
import nltk


In [4]:
filepath = "Dataset/Training/train_data"
df = pd.read_csv(filepath)
df.head()

,Value
0,yep anyway you see where i am coming from many...
1,this is where united nations could come and ma...
2,you kinda sound like johnny bravo saying that.
3,hope you had a good time however possible here...
4,feeling proud bois.


In [8]:
processed_corpus = {} 
sentence_id = 0
for sentence in df['Value']:
    tokenized_sentence = []
    tokens = nltk.word_tokenize(sentence)
    tokens = (token for token in tokens if token.isalpha())
    tokenized_sentence += tokens
    processed_corpus[sentence_id] = tokenized_sentence
    sentence_id += 1


['yep', 'anyway', 'you', 'see', 'where', 'i', 'am', 'coming', 'from', 'many', 'of', 'us', 'are', 'tired', 'of', 'old', 'animosities', 'and', 'want', 'to', 'relate', 'on', 'a', 'personal', 'level', 'not', 'on', 'a', 'political', 'level', 'more', 'genuine', 'that', 'way']


In [9]:
print(processed_corpus[0])

['yep', 'anyway', 'you', 'see', 'where', 'i', 'am', 'coming', 'from', 'many', 'of', 'us', 'are', 'tired', 'of', 'old', 'animosities', 'and', 'want', 'to', 'relate', 'on', 'a', 'personal', 'level', 'not', 'on', 'a', 'political', 'level', 'more', 'genuine', 'that', 'way']


In [ ]:
unigram_counts = {}
for id in processed_corpus:
    for token in processed_corpus[id]:
        unigram_counts[token] += 1
